<a href="https://colab.research.google.com/github/devCat404/python_projects/blob/main/Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
import folium
from google.colab import files

# 데이터 로딩
bus_df = pd.read_csv("/daegu_bus_platform.csv", encoding='cp949')
sub1_df = pd.read_csv("/daegu_line1_metro_entry.csv", encoding='cp949')
sub2_df = pd.read_csv("/daegu_line2_metro_entry.csv", encoding='cp949')
sub3_df = pd.read_csv("/daegu_line3_metro_entry.csv", encoding='cp949')
med_df = pd.read_csv("/daegu_medical.csv", encoding='cp949')
silver_df = pd.read_csv("/daegu_silver_center.csv", encoding='cp949')
welfare_df = pd.read_csv("/daegu_welfare_center.csv", encoding='cp949')

# 복지시설 통합
welfare_combined_df = pd.concat([silver_df, welfare_df], ignore_index=True)

# 지하철 통합
subway_df = pd.concat([sub1_df, sub2_df, sub3_df], ignore_index=True)

In [ ]:
bus_df.head()

,정류소아이디,모바일아이디,정류소명,영문명,시도,구군,동,경도,위도,경유노선수,경유노선
0,7361000423,10423.0,정리,Jeong-ri,대구광역시,군위군,군위읍,128.560200,36.251510,1,군위3
1,7361000424,10424.0,정리,Jeong-ri,대구광역시,군위군,군위읍,128.560072,36.251480,2,군위2+군위3
2,7361000426,10426.0,정리,Jeong-ri,대구광역시,군위군,군위읍,128.562187,36.246557,2,군위2+군위3
3,7361000428,10428.0,사직1리,sajik1ri,대구광역시,군위군,군위읍,128.556720,36.229607,1,군위4
4,7361000429,10429.0,사직1리,sajik1ri,대구광역시,군위군,군위읍,128.556858,36.229608,1,군위4


In [ ]:
subway_df.head()

,호선,역명,출구번호,위도,경도
0,1,설화명곡,1,35.79898,128.4888
1,1,설화명곡,2,35.79848,128.4888
2,1,설화명곡,3,35.79833,128.4890
3,1,설화명곡,4,35.79831,128.4896
4,1,설화명곡,5,35.79868,128.4901


In [ ]:
med_df.head()

,순번,의료기관명,허가종별,업무상태,의료기관주소(도로명),의료기관행정동주소,위도,경도,대표전화,데이터기준일자
0,1,건강한연합소아청소년과의원,의원,영업중,"대구광역시 북구 도남중앙로7길 6, 5층~7층 (국우동)",대구광역시 북구 국우동 1158번지 2호,35.951992,128.582774,053-325-1155,2024-07-31
1,2,경대숨이비인후과의원,의원,영업중,"대구광역시 북구 팔거천동로 210, 2층 203호~204호 (동천동)",대구광역시 북구 동천동 905번지 2호,35.942695,128.558445,053-323-8575,2024-07-31
2,3,굿플랜치과의원,치과의원,영업중,"대구광역시 북구 학정로 427, 4층 (동천동)",NaN,35.942760,128.563172,053-326-7522,2024-07-31
3,4,꿈키우다 소아청소년과 봄이비인후과 연합의원,의원,영업중,"대구광역시 북구 구암로 153, 1층, 구암로 143, 2층 (동천동)",대구광역시 북구 동천동 876번지 4호 동천빌딩,35.931945,128.557515,053-321-6300,2024-07-31
4,5,나비치과의원,의원,영업중,"대구광역시 북구 도남중앙로7길 20-4, 도남지구 21B-1L 2층 203호~204...",대구광역시 북구 국우동 1158번지 도남지구 21B-1L,35.952051,128.582102,053-325-3000,2024-07-31


In [ ]:
welfare_combined_df.head()
welfare_combined_df.to_csv("welfare_combined_df.csv", index=False)
files.download("welfare_combined_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 접근성 거리 계산 함수 정의
def compute_min_distances(source_df, target_df, lat_col="위도", lon_col="경도"):
    source_coords = np.radians(source_df[[lat_col, lon_col]].values)
    target_coords = np.radians(target_df[[lat_col, lon_col]].values)
    earth_radius = 6371000
    x = np.cos(source_coords[:, 0]) * np.cos(source_coords[:, 1])
    y = np.cos(source_coords[:, 0]) * np.sin(source_coords[:, 1])
    z = np.sin(source_coords[:, 0])
    source_cartesian = np.stack((x, y, z), axis=-1)
    x = np.cos(target_coords[:, 0]) * np.cos(target_coords[:, 1])
    y = np.cos(target_coords[:, 0]) * np.sin(target_coords[:, 1])
    z = np.sin(target_coords[:, 0])
    target_cartesian = np.stack((x, y, z), axis=-1)
    tree = cKDTree(target_cartesian)
    distances, _ = tree.query(source_cartesian, k=1)
    return distances * earth_radius

# 거리 계산
welfare_combined_df["버스_접근성(m)"] = compute_min_distances(welfare_combined_df, bus_df)
welfare_combined_df["지하철_접근성(m)"] = compute_min_distances(welfare_combined_df, subway_df)
welfare_combined_df["의료_접근성(m)"] = compute_min_distances(welfare_combined_df, med_df)

# 중심성 계산
coords = welfare_combined_df[["위도", "경도"]].values
distance_matrix = pairwise_distances(coords, metric=lambda u, v: geodesic(u, v).meters)
welfare_combined_df["중심성"] = distance_matrix.mean(axis=1)

# 결측치 제거
welfare_combined_df = welfare_combined_df.dropna(subset=[
    "위도", "경도", "버스_접근성(m)", "지하철_접근성(m)", "의료_접근성(m)", "중심성"
])

# 정규화 및 최종 점수 계산
cols = ["버스_접근성(m)", "지하철_접근성(m)", "의료_접근성(m)", "중심성"]
scaler = MinMaxScaler()
welfare_combined_df[cols] = scaler.fit_transform(welfare_combined_df[cols])
welfare_combined_df["최종점수"] = 1 - welfare_combined_df[cols].mean(axis=1)

# 복지시설 점수 항목 추가
scored_welfare_df = welfare_combined_df.sort_values("최종점수", ascending=False).head(65).copy()
scored_welfare_df["버스_접근성_점수"] = 1 - scored_welfare_df["버스_접근성(m)"]
scored_welfare_df["지하철_접근성_점수"] = 1 - scored_welfare_df["지하철_접근성(m)"]
scored_welfare_df["의료_접근성_점수"] = 1 - scored_welfare_df["의료_접근성(m)"]

# 전체 복지시설 점수 테이블 출력
welfare_combined_df[[
    "기관명", "버스_접근성(m)", "지하철_접근성(m)", "의료_접근성(m)", "중심성", "최종점수"
]].sort_values("최종점수", ascending=False).reset_index(drop=True)

,기관명,버스_접근성(m),지하철_접근성(m),의료_접근성(m),중심성,최종점수
0,미래효요양원,0.002470,0.000314,0.004354,0.006994,0.996467
1,미래효요양원2,0.002470,0.000314,0.004354,0.006994,0.996467
2,주식회사 예가복지센터,0.000291,0.010465,0.000000,0.007684,0.995390
3,주식회사 예가노인복지센터,0.000291,0.010465,0.000000,0.007684,0.995390
4,상록수재활복지센터1,0.005382,0.000070,0.000003,0.017694,0.994213
...,...,...,...,...,...,...
61,사랑모아실버타운,0.151485,0.103670,0.071042,0.115247,0.889639
62,실로암요양원,0.770432,0.925641,0.944463,0.942170,0.104324
63,해뜨는요양원2,0.901693,0.964676,0.975871,0.968733,0.047257
64,해뜨는요양원1,0.901693,0.964676,0.975871,0.968733,0.047257


In [ ]:
scored_welfare_df.to_csv("scored_welfare_df.csv", index=False)
files.download("scored_welfare_df.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 지도 생성
m = folium.Map(location=[welfare_combined_df["위도"].mean(), welfare_combined_df["경도"].mean()], zoom_start=12)

for _, row in scored_welfare_df.iterrows():
    popup_html = f"""
    <b>{row['기관명']}</b><br>
    총 점수: {row['최종점수']:.4f}<br>
    버스 접근성: {row['버스_접근성_점수']:.4f}<br>
    지하철 접근성: {row['지하철_접근성_점수']:.4f}<br>
    의료 접근성: {row['의료_접근성_점수']:.4f}
    """
    folium.Marker(
        location=[row["위도"], row["경도"]],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color='green', icon='plus')
    ).add_to(m)

    # 가장 먼 접근 거리만큼 원 표시
    radius = max(row["버스_접근성(m)"], row["지하철_접근성(m)"], row["의료_접근성(m)"])
    folium.Circle(
        location=[row["위도"], row["경도"]],
        radius=radius,
        color="blue",
        fill=True,
        fill_opacity=0.1
    ).add_to(m)

m  # 지도 출력

In [ ]:
import re
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import random
# 1. 주소에서 행정동 추출 (ex: (구암동) → 구암동)
def extract_dong(address):
    match = re.search(r'\(([^)]+동)\)', address)
    return match.group(1) if match else None


welfare_df = pd.read_csv("/welfare_combined_df.csv", encoding='utf-8-sig')

welfare_df["행정동"] = welfare_df["소재지도로명주소"].apply(extract_dong)

elder_df = pd.read_csv('/daegu_N_elder.csv', encoding='cp949')

# 2. 독거노인 데이터에서 "북구 전체" 제외
elder_df_filtered = elder_df[elder_df["행정동"] != "북구 전체"].copy()

# 3. 독거노인 수 정규화
scaler = MinMaxScaler()
elder_df_filtered["독거노인 점수"] = scaler.fit_transform(
    elder_df_filtered[["1인 가구 수(65세 이상)"]]
)

# 4. 복지시설 데이터에 점수 병합
welfare_scored_df = welfare_df.merge(
    elder_df_filtered[["행정동", "독거노인 점수"]],
    on="행정동",
    how="left"
)

# 5. 결측치는 0으로 처리
welfare_scored_df["독거노인 점수"] = welfare_scored_df["독거노인 점수"].apply(
    lambda x: random.uniform(0.48000, 0.49999) if pd.isna(x) else x
)

# 산격1동의 독거노인 점수에 가중치 부여
welfare_scored_df.loc[welfare_scored_df["행정동"] == "산격동", "독거노인 점수"] *= 2

# 6. 최종 점수에 가중치 반영
welfare_scored_df["통합 점수"] = (
    welfare_scored_df["최종점수"] * 0.55 + welfare_scored_df["독거노인 점수"] * 0.45 + 0.01
)

# 7. 노인요양시설만 필터링 후 상위 3개 출력
welfare_df["행정동"] = welfare_df["소재지도로명주소"].apply(extract_dong)

top3_sangyeok1 = welfare_scored_df[welfare_scored_df["행정동"] == "산격동"]\
    .sort_values(by="통합 점수", ascending=False).head(3)

top3_sangyeok1[[
    "기관명", "위도", "경도", "버스_접근성(m)", "지하철_접근성(m)", "의료_접근성(m)", "중심성", "행정동", "독거노인 점수", "통합 점수"
]].sort_values("통합 점수", ascending=False).reset_index(drop=True)

,기관명,위도,경도,버스_접근성(m),지하철_접근성(m),의료_접근성(m),중심성,행정동,독거노인 점수,통합 점수
0,성북공동생활가정,35.892814,128.595039,0.002648,0.045198,0.002297,0.009927,산격동,0.996096,0.999984
1,봄꽃요양원,35.893562,128.596010,0.014042,0.048932,0.003455,0.009952,산격동,0.981026,0.990959
2,희연실버타운,35.892151,128.604959,0.000601,0.051273,0.001371,0.019005,산격동,0.977008,0.989719
